In [ ]:
import sys
import numpy as np
import pickle
import re
from nltk.stem import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv("stuExampleList.csv", sep=";")
df

,ID,Название СТЕ,Ссылка на изображение,ID конечной категории Портала,Наименование конечной категории Портала,Модель,Производитель
0,37889788,"Электроды-накладки Carto 3 Reference Patches, ...",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793382520,РАСХОДНЫЕ МАТЕРИАЛЫ ДЛЯ СИСТЕМЫ ТРЕХМЕРНОЙ НЕФ...,Webster,"Biosense Webster, Inc."
1,37889786,"Крем-мыло ""Mr. Чистоделоff"" ""Чайный лист"" с гл...",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793302204,Мыло туалетное жидкое,"Крем-мыло ""Mr. Чистоделоff"" ""Чайный лист"" с гл...","ООО ""СВЗХ"""
2,37889778,"Проводник Hi-Torque Эбботт Васкуляр 1005357H, ...",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793403090,ПРОВОДНИК ДЛЯ ДОСТУПА К ПЕРИФЕРИЧЕСКИМ СОСУДАМ...,Hi-Torque,Эбботт Васкуляр
3,37889777,Валик малярный Микрофибра 200мм с рукояткой,https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793372302,Инструмент ручной прочий,Микрофибра,STAYER
4,37889772,Кисть малярная 100 мм HEADMAN,https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793372566,Кисти и щетки малярные,Бейсик,HEADMAN
...,...,...,...,...,...,...,...
899995,36008423,"Стул ""КОЖПРОММЕБЕЛЬ"" экокожа 495х790мм черный",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793334102,Стулья на металлическом каркасе,СП,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОЖП..."
899996,36008422,Винт с полукруглой головкой М5х50,https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793371158,"Метизы, крепежные изделия прочие",М5,ООО «Столичный крепеж»
899997,36008421,"Кресло для посетителя ШхГхВ: 600х450х930 мм, ч...",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793358431,Кресла для зрительных залов,самба,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КОЖП..."
899998,36008420,"Кроссовки женские NB, черные, размер 10",https://zakupki.mos.ru/newapi/api/Core/Thumbna...,793370606,Кроссовки,Fresh Foam,New Balance


In [ ]:
counter = df['ID конечной категории Портала'].value_counts()
df["ctg_cnt"] = df['ID конечной категории Портала'].progress_apply(lambda x: counter[x])

100%|██████████| 900000/900000 [00:05<00:00, 170504.01it/s]


In [ ]:
df = df[df.ctg_cnt > 122]

In [ ]:
df['ID конечной категории Портала'].value_counts()

ID конечной категории Портала
793369656    64019
793374776    57712
793362679    38471
793369662    32891
793373176    32413
             ...  
793374390      124
793358157      124
793364206      123
793374400      123
793373518      123
Name: count, Length: 830, dtype: int64

In [ ]:
stats = df.groupby('ID конечной категории Портала').progress_apply(lambda x: x.sample(min(len(x), 2000)))

100%|██████████| 830/830 [00:01<00:00, 705.75it/s] 


In [ ]:
new_df = stats.rename(columns={'ID конечной категории Портала': "reset"}).reset_index()

In [ ]:
new_df['ID конечной категории Портала'].value_counts().describe()

count     830.000000
mean      454.281928
std       483.213305
min       123.000000
25%       168.250000
50%       258.000000
75%       483.000000
max      2000.000000
Name: count, dtype: float64

In [ ]:
new_df['Наименование конечной категории Портала'].values

array(['Конверты почтовые', 'Конверты почтовые', 'Конверты почтовые', ...,
       'КАТЕТЕРЫ ДЛЯ ГЕМОДИАЛИЗА', 'КАТЕТЕРЫ ДЛЯ ГЕМОДИАЛИЗА',
       'КАТЕТЕРЫ ДЛЯ ГЕМОДИАЛИЗА'], dtype=object)

In [ ]:
label, index = pd.factorize(new_df['Наименование конечной категории Портала'])
new_df["label"] = label

In [ ]:
stemmer = SnowballStemmer("russian")
tokenizer = RegexpTokenizer(r'\w+')

def clear_text(text):
    return ' '.join(stemmer.stem(x) for x in tokenizer.tokenize(text.lower()))

In [ ]:
new_df["feature"] = new_df["Название СТЕ"].progress_apply(clear_text)

100%|██████████| 377054/377054 [02:44<00:00, 2291.13it/s]


In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SGDClassifier(loss='hinge')),
])

In [ ]:
text_clf.fit(new_df["feature"], new_df["label"])

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', SGDClassifier())])

In [ ]:
text_clf.predict([clear_text("Мусор")])

array([244])

In [ ]:
index[244]

'Мешки полимерные'

In [ ]:
import pickle

with open("index.pkl", "wb") as f:
    pickle.dump(list(index), f)

In [ ]:
!cp index.pkl /content/drive/MyDrive/index.pkl

In [ ]:
with open("index.pkl", "rb") as f:
    print(pickle.load(f))

['Конверты почтовые', 'Наградная продукция', 'Конфеты', 'Зеркала интерьерные', 'Наборы продовольственные', 'Витрины', 'Скамьи', 'Банкетки', 'Тенты', 'Комплексные пищевые добавки', 'Материалы расходные для учебного оборудования', 'Оборудование для уроков труда', 'Сервер', 'Программное обеспечение прикладное для автоматизированных рабочих мест (кроме заказного)', 'Программное обеспечение специализированное для защиты информации', 'Кофе жареный', 'Печенье сладкое', 'Стойки сценические', 'Струны для музыкальных инструментов', 'Принадлежности для музыкальных инструментов прочие', 'Декорации', 'Концентраторы USB', 'Чай черный (ферментированный)', 'Приборы учета тепловой энергии', 'Приборы учета прочие', 'Одежда для взрослых форменная', 'Платья для взрослых', 'Жилеты для взрослых', 'Халаты для взрослых', 'Футболки трикотажные для взрослых', 'Аксессуары', 'Платья для девочек', 'Футболки трикотажные для детей', 'Толстовки с длинным рукавом (фуфайки) для детей', 'Одежда для детей и подростков фо